代码来源：https://github.com/bentrevett/pytorch-nli

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchtext.legacy import data,datasets

import random
import numpy as np

import time

from tqdm import tqdm

In [2]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [3]:
TEXT = data.Field(tokenize = 'spacy', tokenizer_language='en_core_web_sm',lower = True)
LABEL = data.LabelField(sequential=False)

In [4]:
train_data, valid_data, test_data = datasets.SNLI.splits(TEXT, LABEL)

In [66]:
train_data.examples=train_data.examples[0:10000]

In [67]:
print(f"Number of training examples: {len(train_data)}")
print(f"Number of validation examples: {len(valid_data)}")
print(f"Number of testing examples: {len(test_data)}")

Number of training examples: 10000
Number of validation examples: 9842
Number of testing examples: 9824


In [68]:
print(vars(train_data.examples[0]))

{'premise': ['a', 'person', 'on', 'a', 'horse', 'jumps', 'over', 'a', 'broken', 'down', 'airplane', '.'], 'hypothesis': ['a', 'person', 'is', 'training', 'his', 'horse', 'for', 'a', 'competition', '.'], 'label': 'neutral'}


In [69]:
MIN_FREQ = 2

TEXT.build_vocab(train_data, 
                 min_freq = MIN_FREQ,
                 vectors = "glove.6B.100d",
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

In [70]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")

Unique tokens in TEXT vocabulary: 3972


In [71]:
print(TEXT.vocab.freqs.most_common(20))

[('a', 25919), ('.', 17395), ('the', 9645), ('in', 7571), ('is', 6374), ('man', 4757), ('on', 4319), ('and', 3882), ('are', 3716), ('of', 3463), ('with', 3043), ('woman', 2654), ('people', 2357), ('two', 2336), (',', 2223), ('to', 2123), ('at', 1990), ('an', 1463), ('his', 1376), ('wearing', 1375)]


In [72]:
BATCH_SIZE = 512

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device)

### BiLSTM





In [73]:
class NLIBiLSTM(nn.Module):
    def __init__(self, 
                 input_dim, 
                 embedding_dim,
                 hidden_dim,
                 n_lstm_layers,
                 n_fc_layers,
                 output_dim, 
                 dropout, 
                 pad_idx):
        
        super().__init__()
                                
        self.embedding = nn.Embedding(input_dim, embedding_dim, padding_idx = pad_idx)
        
        self.translation = nn.Linear(embedding_dim, hidden_dim)
        
        self.lstm = nn.LSTM(hidden_dim, 
                            hidden_dim, 
                            num_layers = n_lstm_layers, 
                            bidirectional = True, 
                            dropout=dropout if n_lstm_layers > 1 else 0)
        
        fc_dim = hidden_dim * 2
        
        fcs = [nn.Linear(fc_dim * 2, fc_dim * 2) for _ in range(n_fc_layers)]
        
        self.fcs = nn.ModuleList(fcs)
        
        self.fc_out = nn.Linear(fc_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, prem, hypo):

        prem_seq_len, batch_size = prem.shape
        hypo_seq_len, _ = hypo.shape
        
        #prem = [prem sent len, batch size]
        #hypo = [hypo sent len, batch size]
        
        embedded_prem = self.embedding(prem)
        embedded_hypo = self.embedding(hypo)
        
        #embedded_prem = [prem sent len, batch size, embedding dim]
        #embedded_hypo = [hypo sent len, batch size, embedding dim]
        
        translated_prem = F.relu(self.translation(embedded_prem))
        translated_hypo = F.relu(self.translation(embedded_hypo))
        
        #translated_prem = [prem sent len, batch size, hidden dim]
        #translated_hypo = [hypo sent len, batch size, hidden dim]
        
        outputs_prem, (hidden_prem, cell_prem) = self.lstm(translated_prem)
        outputs_hypo, (hidden_hypo, cell_hypo) = self.lstm(translated_hypo)

        #outputs_x = [sent len, batch size, n directions * hid dim]
        #hidden_x = [n layers * n directions, batch size, hid dim]
        #cell_x = [n layers * n directions, batch size, hid dim]
        
        hidden_prem = torch.cat((hidden_prem[-1], hidden_prem[-2]), dim=-1)
        hidden_hypo = torch.cat((hidden_hypo[-1], hidden_hypo[-2]), dim=-1)
        
        #hidden_x = [batch size, fc dim]

        hidden = torch.cat((hidden_prem, hidden_hypo), dim=1)

        #hidden = [batch size, fc dim * 2]
        
        # 不要线性层
        # for fc in self.fcs:
        #     hidden = fc(hidden)
        #     hidden = F.relu(hidden)
        #     hidden = self.dropout(hidden)
        
        prediction = self.fc_out(hidden)
        
        #prediction = [batch size, output dim]
        
        return prediction

In [96]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 300
N_LSTM_LAYERS = 2
N_FC_LAYERS = 3
OUTPUT_DIM = len(LABEL.vocab)
DROPOUT = 0.25
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = NLIBiLSTM(INPUT_DIM,
                  EMBEDDING_DIM,
                  HIDDEN_DIM,
                  N_LSTM_LAYERS,
                  N_FC_LAYERS,
                  OUTPUT_DIM,
                  DROPOUT,
                  PAD_IDX).to(device)

In [75]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 8,364,303 trainable parameters


In [76]:
pretrained_embeddings = TEXT.vocab.vectors
print(pretrained_embeddings.shape)

torch.Size([3972, 100])


In [77]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[ 1.4816,  0.7536,  0.4595,  ...,  1.0832, -1.1331,  0.5280],
        [ 0.8705, -0.7628, -0.6878,  ...,  0.0246,  2.7670, -0.5691],
        [-0.2709,  0.0440, -0.0203,  ..., -0.4923,  0.6369,  0.2364],
        ...,
        [-0.1194,  0.5419,  0.6217,  ..., -0.3805, -0.0422,  0.3516],
        [-0.1184, -2.4061,  0.2519,  ...,  0.6506, -0.7514,  0.8046],
        [-0.1509,  0.2089,  0.7544,  ..., -0.2359,  0.4850,  0.0263]],
       device='cuda:0')

In [78]:
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 1.4816,  0.7536,  0.4595,  ...,  1.0832, -1.1331,  0.5280],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.2709,  0.0440, -0.0203,  ..., -0.4923,  0.6369,  0.2364],
        ...,
        [-0.1194,  0.5419,  0.6217,  ..., -0.3805, -0.0422,  0.3516],
        [-0.1184, -2.4061,  0.2519,  ...,  0.6506, -0.7514,  0.8046],
        [-0.1509,  0.2089,  0.7544,  ..., -0.2359,  0.4850,  0.0263]],
       device='cuda:0')


In [79]:
model.embedding.weight.requires_grad = False

In [80]:
print(model)

NLIBiLSTM(
  (embedding): Embedding(3972, 100, padding_idx=1)
  (translation): Linear(in_features=100, out_features=300, bias=True)
  (lstm): LSTM(300, 300, num_layers=2, dropout=0.25, bidirectional=True)
  (fcs): ModuleList(
    (0): Linear(in_features=1200, out_features=1200, bias=True)
    (1): Linear(in_features=1200, out_features=1200, bias=True)
    (2): Linear(in_features=1200, out_features=1200, bias=True)
  )
  (fc_out): Linear(in_features=1200, out_features=3, bias=True)
  (dropout): Dropout(p=0.25, inplace=False)
)


In [81]:
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss().to(device)


In [82]:
def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    correct = max_preds.squeeze(1).eq(y)
    correct=torch.tensor(correct.sum()).to(device)
    all=torch.tensor(y.shape[0]).to(device)

    return correct/ all

In [83]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in tqdm(iterator):
        
        prem = batch.premise
        hypo = batch.hypothesis
        labels = batch.label
        
        optimizer.zero_grad()
        
        #prem = [prem sent len, batch size]
        #hypo = [hypo sent len, batch size]
        
        predictions = model(prem, hypo)
        

        #predictions = [batch size, output dim]
        #labels = [batch size]
        
        loss = criterion(predictions, labels)
                
        acc = categorical_accuracy(predictions, labels)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [84]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in tqdm(iterator):

            prem = batch.premise
            hypo = batch.hypothesis
            labels = batch.label
                        
            predictions = model(prem, hypo)
            
            loss = criterion(predictions, labels)
                
            acc = categorical_accuracy(predictions, labels)
            
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [85]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [86]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

  0%|          | 0/20 [00:00<?, ?it/s]C:\Users\zoro\AppData\Local\Temp/ipykernel_9672/2499469631.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  correct=torch.tensor(correct.sum()).to(device)
100%|██████████| 20/20 [00:00<00:00, 21.13it/s]


Epoch: 01 | Epoch Time: 0m 11s
	Train Loss: 1.064 | Train Acc: 41.23%
	 Val. Loss: 1.025 |  Val. Acc: 48.37%


100%|██████████| 20/20 [00:00<00:00, 21.09it/s]


Epoch: 02 | Epoch Time: 0m 10s
	Train Loss: 0.983 | Train Acc: 52.10%
	 Val. Loss: 0.983 |  Val. Acc: 52.64%


100%|██████████| 20/20 [00:00<00:00, 21.11it/s]


Epoch: 03 | Epoch Time: 0m 10s
	Train Loss: 0.933 | Train Acc: 55.82%
	 Val. Loss: 0.950 |  Val. Acc: 54.50%


100%|██████████| 20/20 [00:00<00:00, 21.42it/s]


Epoch: 04 | Epoch Time: 0m 10s
	Train Loss: 0.898 | Train Acc: 58.69%
	 Val. Loss: 0.930 |  Val. Acc: 55.69%


100%|██████████| 20/20 [00:00<00:00, 20.97it/s]

Epoch: 05 | Epoch Time: 0m 10s
	Train Loss: 0.869 | Train Acc: 60.58%
	 Val. Loss: 0.913 |  Val. Acc: 56.49%


In [87]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} |  Test Acc: {test_acc*100:.2f}%')

  0%|          | 0/20 [00:00<?, ?it/s]C:\Users\zoro\AppData\Local\Temp/ipykernel_9672/2499469631.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  correct=torch.tensor(correct.sum()).to(device)
100%|██████████| 20/20 [00:00<00:00, 20.63it/s]

Test Loss: 0.914 |  Test Acc: 56.97%


In [88]:
def predict_inference(premise, hypothesis, text_field, label_field, model, device):
    
    model.eval()
    
    if isinstance(premise, str):
        premise = text_field.tokenize(premise)
    
    if isinstance(hypothesis, str):
        hypothesis = text_field.tokenize(hypothesis)
    
    if text_field.lower:
        premise = [t.lower() for t in premise]
        hypothesis = [t.lower() for t in hypothesis]
        
    premise = [text_field.vocab.stoi[t] for t in premise]
    hypothesis = [text_field.vocab.stoi[t] for t in hypothesis]
    
    premise = torch.LongTensor(premise).unsqueeze(1).to(device)
    hypothesis = torch.LongTensor(hypothesis).unsqueeze(1).to(device)
    
    prediction = model(premise, hypothesis)
    
    prediction = prediction.argmax(dim=-1).item()
    
    return label_field.vocab.itos[prediction]

In [95]:
premise = 'a man sitting on a green bench.'
hypothesis = 'a woman sitting on a green bench.'
predict_inference(premise, hypothesis, TEXT, LABEL, model, device)

'contradiction'